<a href="https://colab.research.google.com/github/sarahfernandadc/spotify_listening_habits_analysis/blob/main/spotify_listening_habits_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# ==============================================================================
# LOADING JSON FILES
# ==============================================================================

# 1. Import necessary libraries
import pandas as pd
import glob
from typing import List, Tuple

def load_and_merge_spotify_history(path_pattern: str) -> pd.DataFrame:
    """
    Loads and merges multiple Spotify streaming history JSON files into a single
    Pandas DataFrame.

    This function searches for files matching a given pattern, reads each JSON file
    into a DataFrame, and concatenates them into one comprehensive DataFrame.

    Parameters:
    -----------
    path_pattern : str
        The path and pattern to match for the JSON files (e.g., 'StreamingHistory_music*.json').

    Returns:
    --------
    pd.DataFrame
        A single DataFrame containing all the combined streaming history data.
        Returns an empty DataFrame if no files are found.
    """
    #print(f"Searching for files with pattern: '{path_pattern}'...")
    file_list: List[str] = glob.glob(path_pattern)

    if not file_list:
        print("Warning: No files found matching the pattern. Please check the file path.")
        return pd.DataFrame() # Return an empty DataFrame if no files are found

    #print(f"Found {len(file_list)} files: {file_list}")

    dataframe_list: List[pd.DataFrame] = []
    #print("\nReading and processing each file...")
    for current_file in file_list:
        try:
            temp_df = pd.read_json(current_file)
            dataframe_list.append(temp_df)
            print(f"  - Successfully loaded {current_file}")
        except Exception as e:
            print(f"  - Error loading {current_file}: {e}")

    #print("\nMerging all data into a single table...")
    full_history_df = pd.concat(dataframe_list, ignore_index=True)
    #print("Merge completed successfully!")

    return full_history_df

In [42]:
# ==============================================================================
# DATA CLEANING
# ==============================================================================

# ------------------------------------------------------------------------------
# Helper Functions (Each function has a single responsibility)
# ------------------------------------------------------------------------------

def _convert_endtime_to_datetime(df: pd.DataFrame) -> pd.DataFrame:
    """Converts the 'endTime' column from object to datetime type."""
    #print("Converting 'endTime' to datetime...")
    df['endTime'] = pd.to_datetime(df['endTime'])
    return df

def _add_time_played_features(df: pd.DataFrame) -> pd.DataFrame:
    """Creates 'secondsPlayed', 'minutesPlayed', 'hoursPlayed', and 'daysPlayed' columns from 'msPlayed'."""
    #print("Creating time played features (seconds, minutes, hours, days)...")
    df['secondsPlayed'] = df['msPlayed'] / 1000
    df['minutesPlayed'] = df['secondsPlayed'] / 60
    df['hoursPlayed'] = df['minutesPlayed'] / 60
    df['daysPlayed'] = df['hoursPlayed'] / 24
    return df

def _check_for_null_values(df: pd.DataFrame) -> pd.DataFrame:
    """Checks for and reports null values in the DataFrame."""
    #print("Checking for and removing null values...")
    null_counts = df.isnull().sum()
    nulls_to_report = null_counts[null_counts > 0]

    if nulls_to_report.empty:
        #print("--> No null values found.")
        return df

    #print("--> Null values found in the following columns:")
    print(nulls_to_report)
    initial_rows = len(df)
    df = df.dropna()
    rows_removed = initial_rows - len(df)
    #print(f"--> {rows_removed} row(s) with null values were removed.")
    return df

# ------------------------------------------------------------------------------
# Main Pipeline Function (This function calls the others in order)
# ------------------------------------------------------------------------------

def preprocess_spotify_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Runs the full data cleaning and preparation pipeline on the raw Spotify history data.

    Parameters:
    -----------
    df : pd.DataFrame
        The raw DataFrame loaded from the JSON files.

    Returns:
    --------
    pd.DataFrame
        A cleaned and prepared DataFrame ready for analysis.
    """
    #print("\n--- Starting Data Preparation Pipeline ---")
    df = (df.pipe(_convert_endtime_to_datetime)
            .pipe(_add_time_played_features)
            .pipe(_check_for_null_values)
    )
    print("\n--- Data Preparation Pipeline Finished! ---")
    return df

In [43]:
# ==============================================================================
# DATAFRAME SEPARATION (LISTENED X SKIPPED)
# ==============================================================================

def separate_listens_from_skips(df: pd.DataFrame, min_seconds_threshold: int = 30) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Separates the DataFrame into two: one for listened tracks and one for skipped tracks.

    Parameters:
    -----------
    df : pd.DataFrame
        The preprocessed DataFrame that still contains all tracks.
    min_seconds_threshold : int, optional
        The duration in seconds to define a skip. Defaults to 30.

    Returns:
    --------
    Tuple[pd.DataFrame, pd.DataFrame]
        A tuple containing two DataFrames: (listened_df, skipped_df).
    """
    #print(f"\n--- Separating Skips vs. Listens (Threshold: {min_seconds_threshold} sec) ---")

    # Create the DataFrame for listened tracks
    listened_df = df[df['secondsPlayed'] > min_seconds_threshold].copy()

    # Create the DataFrame for skipped tracks
    skipped_df = df[df['secondsPlayed'] <= min_seconds_threshold].copy()

    print(f"\n--> Main analysis set (listened_df) created with {len(listened_df)} tracks.")
    print(f"--> Skips analysis set (skipped_df) created with {len(skipped_df)} tracks.")

    return listened_df, skipped_df

In [48]:
def calculate_top_tracks(df: pd.DataFrame, top_value: int = 10) -> pd.DataFrame:
    """
    Calculates the top `top_value` most listened tracks from the DataFrame based on the count
    of times each track was played.

    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame containing the listened tracks data. Assumes it has
        'trackName' and 'artistName' columns.
    top_value : int, optional
        The number of top tracks to return. Must be a positive integer. Defaults to 10.

    Returns:
    --------
    pd.DataFrame
        A DataFrame with the top `top_value` tracks, showing 'trackName', 'artistName',
        and 'listenCount', sorted in descending order of listen count.
        Returns an empty DataFrame if the input DataFrame is empty.
    """
    #print("\n--- Calculating Top 10 Most Listened Tracks ---")

    if df.empty:
        print("Warning: Input DataFrame is empty. Cannot calculate top tracks.")
        return pd.DataFrame()

    if not isinstance(top_value, int) or top_value <= 0:
        raise ValueError("Parameter 'top_value' must be a positive integer.")

    # Group by track name and artist name and count the occurrences
    top_tracks = (
        df.groupby(['trackName', 'artistName'])
          .size()
          .reset_index(name='listenCount')
          .sort_values(by=['listenCount', 'trackName', 'artistName'], ascending=[False, True, True])
          .head(top_value)
    )

    print("Top 10 tracks calculated successfully.")

    return top_tracks

In [50]:
# ==============================================================================
# MAIN EXECUTION BLOCK
# ==============================================================================

if __name__ == "__main__":

    # Define constants
    SPOTIFY_FILE_PATTERN = 'StreamingHistory*.json'
    MIN_SECONDS_FOR_LISTEN = 5

    # Load Data ---
    raw_df = load_and_merge_spotify_history(path_pattern=SPOTIFY_FILE_PATTERN)

    if not raw_df.empty:
        # Preprocess Data ---
        preprocessed_df = preprocess_spotify_data(raw_df)

        # Separate Data ---
        listened_df, skipped_df = separate_listens_from_skips(
            preprocessed_df,
            min_seconds_threshold=MIN_SECONDS_FOR_LISTEN
        )

        if not listened_df.empty:
            top_tracks_df = calculate_top_tracks(listened_df, top_value=5)
        else:
            print("'listened_df' is empty. No tracks to analyze for top plays.")

        print(top_tracks_df)


  - Successfully loaded StreamingHistory_music_2.json
  - Successfully loaded StreamingHistory_music_1.json
  - Successfully loaded StreamingHistory_music_0.json

--- Data Preparation Pipeline Finished! ---

--> Main analysis set (listened_df) created with 21921 tracks.
--> Skips analysis set (skipped_df) created with 6472 tracks.
Top 10 tracks calculated successfully.
                             trackName     artistName  listenCount
2564                    Seja Bem-Vinda          Niink          222
2116                 Os Menino da Nova  Supernova Ent          182
1812                  Mine - Live/2011   Taylor Swift          178
812   Devolve as Correntes / Invisível          Veigh          140
3082         VADIA (feat. Veigh & G.A)  Supernova Ent          138
